# Red Neuronal

Variables caracteristicas

Fuente_Industrial (Discreta: Baja/Alta)
Velocidad_Corriente (Discreta: Lenta/Rápida)
Profundidad (Discreta: Superficie/Fondo)
Concentracion_Mercurio (Discreta: Baja/Media/Alta)
Impacto_Ecologico (Binaria: Sí/No)

Variable objetivo: 
Riesgo_Humano (Binaria: Seguro/Peligroso)


* Configurar Red neuronal para predecir el riesgo humano, capa de entrada, una capa oculta y una capa de salida
* Es apropiado usar Softmax??
* Configure capa oculta sigmoide | RElU
* Comapre los modelos experimentados con base a metricas de precisión, exactitud, exhaustividad.


Utilizar datos_mercurio_seguros.csv para llenar la tabla de distribucion de probabilidades.

Consultas a responder con el modelo:

* Probabilidad de alto riesgo humano cuando la fuente industrial es alta
* Cuál es la probabilidad de que el riesgo humano sea peligroso cuando la concentración de mercurio es alta y la fuente industrial está activa?"
* ¿Cómo cambia el impacto ecológico si se reduce la velocidad de la corriente en zonas profundas?"
* "¿A partir de qué nivel de concentración la probabilidad de riesgo humano supera el 50%?"

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from RedNeuronal import RedNeuronal, Sigmoid, ReLU, Softmax

In [2]:
# =================================================================
# 1. CARGAR Y PREPROCESAR DATOS
# =================================================================
# Cargar datos
data = pd.read_csv('datos_mercurio_seguros.csv')

# Mapear valores numéricos a etiquetas textuales
mapping = {
    'Fuente_Industrial': {0: 'Baja', 1: 'Alta'},
    'Velocidad_Corriente': {0: 'Lenta', 1: 'Rápida'},
    'Profundidad': {0: 'Superficie', 1: 'Fondo'},
    'Concentracion_Mercurio': {0: 'Baja', 1: 'Media', 2: 'Alta'},
    'Impacto_Ecologico': {0: 'No', 1: 'Sí'},
    'Riesgo_Humano': {0: 'Seguro', 1: 'Peligroso'}
}

# Aplicar mapeo
for col, mapping_dict in mapping.items():
    if col in data.columns:
        data[col] = data[col].map(mapping_dict)

# Codificar características usando One-Hot Encoding
categorical_cols = [
    'Fuente_Industrial', 
    'Velocidad_Corriente', 
    'Profundidad', 
    'Concentracion_Mercurio', 
    'Impacto_Ecologico'
]

encoder = OneHotEncoder(sparse_output=False, drop='if_binary')
X_encoded = encoder.fit_transform(data[categorical_cols])

# Obtener nombres de características después de la codificación
feature_names = []
for i, col in enumerate(categorical_cols):
    if len(encoder.categories_[i]) == 2:  # Características binarias
        feature_names.append(f"{col}_{encoder.categories_[i][1]}")
    else:
        for category in encoder.categories_[i]:
            feature_names.append(f"{col}_{category}")

# Variable objetivo
y = data['Riesgo_Humano'].map({'Seguro': 0, 'Peligroso': 1}).values

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.3, random_state=42
)

In [3]:
# =================================================================
# 2. CONFIGURAR Y ENTRENAR MODELOS
# =================================================================
# Configuración común
input_size = X_train.shape[1]
output_size = 2  # Seguro (0) y Peligroso (1)

# Modelo 1: Capa oculta con activación Sigmoid
model_sigmoid = RedNeuronal(
    output_activation=Softmax(),
    hidden_activation=Sigmoid(),
    hidden_layers=1,
    lambda_=0.01
)
model_sigmoid.configurar_capas(
    input_size=input_size,
    hidden_size=10,  # 10 neuronas en capa oculta
    output_size=output_size
)
model_sigmoid.inicializar_parametros()
model_sigmoid.fit(X_train, y_train)
result_sigmoid = model_sigmoid.entrenar(max_iter=200)

# Modelo 2: Capa oculta con activación ReLU
model_relu = RedNeuronal(
    output_activation=Softmax(),
    hidden_activation=ReLU(),
    hidden_layers=1,
    lambda_=0.01
)
model_relu.configurar_capas(
    input_size=input_size,
    hidden_size=10,  # 10 neuronas en capa oculta
    output_size=output_size
)
model_relu.inicializar_parametros()
model_relu.fit(X_train, y_train)
result_relu = model_relu.entrenar(max_iter=200)

In [4]:
# =================================================================
# 3. EVALUAR MODELOS
# =================================================================
def evaluar_modelo(modelo, X, y, nombre):
    metricas = modelo.metricas_completas(X, y)
    
    print(f"\n🔍 Métricas para modelo {nombre}:")
    print(f"• Accuracy: {metricas['accuracy']:.4f}")
    print(f"• Precision (Clase 0 - Seguro): {metricas['precision'][0]:.4f}")
    print(f"• Precision (Clase 1 - Peligroso): {metricas['precision'][1]:.4f}")
    print(f"• Recall (Clase 0 - Seguro): {metricas['recall'][0]:.4f}")
    print(f"• Recall (Clase 1 - Peligroso): {metricas['recall'][1]:.4f}")
    print(f"• F1-Score (Clase 0 - Seguro): {metricas['f1'][0]:.4f}")
    print(f"• F1-Score (Clase 1 - Peligroso): {metricas['f1'][1]:.4f}")
    
    return metricas

# Evaluar ambos modelos en datos de prueba
print("\n" + "="*50)
metricas_sigmoid = evaluar_modelo(model_sigmoid, X_test, y_test, "Sigmoid")
print("\n" + "="*50)
metricas_relu = evaluar_modelo(model_relu, X_test, y_test, "ReLU")
print("="*50)



🔍 Métricas para modelo Sigmoid:
• Accuracy: 0.7625
• Precision (Clase 0 - Seguro): 0.7625
• Precision (Clase 1 - Peligroso): 0.0000
• Recall (Clase 0 - Seguro): 1.0000
• Recall (Clase 1 - Peligroso): 0.0000
• F1-Score (Clase 0 - Seguro): 0.8652
• F1-Score (Clase 1 - Peligroso): 0.0000


🔍 Métricas para modelo ReLU:
• Accuracy: 0.4458
• Precision (Clase 0 - Seguro): 0.7358
• Precision (Clase 1 - Peligroso): 0.2164
• Recall (Clase 0 - Seguro): 0.4262
• Recall (Clase 1 - Peligroso): 0.5088
• F1-Score (Clase 0 - Seguro): 0.5398
• F1-Score (Clase 1 - Peligroso): 0.3037


In [5]:
# =================================================================
# 4. COMPARAR MODELOS
# =================================================================
print("\n🏆 Comparación de modelos:")
print(f"• Accuracy - Sigmoid: {metricas_sigmoid['accuracy']:.4f} | ReLU: {metricas_relu['accuracy']:.4f}")
print(f"• F1-Score (Peligroso) - Sigmoid: {metricas_sigmoid['f1'][1]:.4f} | ReLU: {metricas_relu['f1'][1]:.4f}")

# Determinar mejor modelo
if metricas_relu['accuracy'] > metricas_sigmoid['accuracy']:
    mejor_modelo = model_relu
    mejor_nombre = "ReLU"
else:
    mejor_modelo = model_sigmoid
    mejor_nombre = "Sigmoid"

print(f"\nEl mejor modelo es: {mejor_nombre}")



🏆 Comparación de modelos:
• Accuracy - Sigmoid: 0.7625 | ReLU: 0.4458
• F1-Score (Peligroso) - Sigmoid: 0.0000 | ReLU: 0.3037

El mejor modelo es: Sigmoid


In [6]:
# =================================================================
# 5. RESPUESTA A CONSULTAS CON EL MEJOR MODELO
# =================================================================
def predecir_riesgo(mejor_modelo, caracteristicas):
    """Predice la probabilidad de riesgo peligroso"""
    # Convertir características a formato one-hot
    entrada = pd.DataFrame([caracteristicas], columns=categorical_cols)
    entrada_encoded = encoder.transform(entrada)
    
    # Predecir probabilidades
    probabilidades = mejor_modelo.predecir_probabilidad(entrada_encoded)
    return probabilidades[0][1]  # Probabilidad de clase Peligroso

print("\n🔮 Consultas con el mejor modelo:")
# Consulta 1: Probabilidad de riesgo peligroso cuando fuente industrial es alta
caracteristicas1 = {
    'Fuente_Industrial': 'Alta',
    'Velocidad_Corriente': 'Lenta',  # Valor por defecto
    'Profundidad': 'Fondo',         # Valor por defecto
    'Concentracion_Mercurio': 'Media',  # Valor por defecto
    'Impacto_Ecologico': 'Sí'       # Valor por defecto
}
prob1 = predecir_riesgo(mejor_modelo, caracteristicas1)
print(f"1. Probabilidad de riesgo peligroso | Fuente Industrial = Alta: {prob1:.4f}")

# Consulta 2: Riesgo peligroso con concentración alta y fuente activa
caracteristicas2 = {
    'Fuente_Industrial': 'Alta',
    'Velocidad_Corriente': 'Lenta',
    'Profundidad': 'Fondo',
    'Concentracion_Mercurio': 'Alta',
    'Impacto_Ecologico': 'Sí'
}
prob2 = predecir_riesgo(mejor_modelo, caracteristicas2)
print(f"2. Probabilidad riesgo peligroso | [Concentración=Alta y Fuente=Alta]: {prob2:.4f}")

# Consulta 3: Cambio al reducir velocidad en zonas profundas
# Estado base: Velocidad rápida en fondo
caracteristicas3_base = {
    'Fuente_Industrial': 'Alta',
    'Velocidad_Corriente': 'Rápida',
    'Profundidad': 'Fondo',
    'Concentracion_Mercurio': 'Media',
    'Impacto_Ecologico': 'Sí'
}

# Estado reducido: Velocidad lenta en fondo
caracteristicas3_reducido = {
    'Fuente_Industrial': 'Alta',
    'Velocidad_Corriente': 'Lenta',
    'Profundidad': 'Fondo',
    'Concentracion_Mercurio': 'Media',
    'Impacto_Ecologico': 'Sí'
}

prob3_base = predecir_riesgo(mejor_modelo, caracteristicas3_base)
prob3_reducido = predecir_riesgo(mejor_modelo, caracteristicas3_reducido)
print(f"3. Cambio al reducir velocidad en zonas profundas:")
print(f"   • Velocidad rápida: P(Riesgo) = {prob3_base:.4f}")
print(f"   • Velocidad lenta: P(Riesgo) = {prob3_reducido:.4f}")
print(f"   • Diferencia: {prob3_reducido - prob3_base:.4f}")

# Consulta 4: Nivel de concentración donde riesgo supera 50%
concentraciones = ['Baja', 'Media', 'Alta']
umbral = 0.5
umbral_superado = False

print("\n4. Probabilidad de riesgo peligroso por nivel de concentración:")
for conc in concentraciones:
    caracteristicas4 = {
        'Fuente_Industrial': 'Alta',
        'Velocidad_Corriente': 'Lenta',
        'Profundidad': 'Fondo',
        'Concentracion_Mercurio': conc,
        'Impacto_Ecologico': 'Sí'
    }
    prob = predecir_riesgo(mejor_modelo, caracteristicas4)
    print(f"   • Concentración {conc}: {prob:.4f} {'(>50%)' if prob > umbral else ''}")
    
    if not umbral_superado and prob > umbral:
        nivel_umbral = conc
        umbral_superado = True

if umbral_superado:
    print(f"\nEl riesgo supera 50% cuando la concentración es: {nivel_umbral}")
else:
    print("\nEl riesgo no supera el 50% en ningún nivel de concentración")


🔮 Consultas con el mejor modelo:
1. Probabilidad de riesgo peligroso | Fuente Industrial = Alta: 0.2157
2. Probabilidad riesgo peligroso | [Concentración=Alta y Fuente=Alta]: 0.2232
3. Cambio al reducir velocidad en zonas profundas:
   • Velocidad rápida: P(Riesgo) = 0.2158
   • Velocidad lenta: P(Riesgo) = 0.2157
   • Diferencia: -0.0001

4. Probabilidad de riesgo peligroso por nivel de concentración:
   • Concentración Baja: 0.2299 
   • Concentración Media: 0.2157 
   • Concentración Alta: 0.2232 

El riesgo no supera el 50% en ningún nivel de concentración
